# 武器データのスクレイピング 
メイン、サブ、スペシャルのテーブルデータの作成



In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

In [2]:
url = "https://www.ikaclo.jp/2/weapons/"
buki_df = pd.read_csv("../../data/Raw/statink-weapon2.csv")

In [3]:
buki_df["[ja-JP]"]

0                .52ガロン
1              .52ガロンデコ
2                .96ガロン
3              .96ガロンデコ
4              プロモデラーMG
5              プロモデラーPG
6              プロモデラーRG
7         ジェットスイーパーカスタム
8              もみじシューター
9           ボトルガイザーフォイル
10         プライムシューターコラボ
11       ヒーローシューター レプリカ
12            ジェットスイーパー
13          .52ガロンベッチュー
14        スプラシューターベッチュー
15             おちばシューター
16       プライムシューターベッチュー
17           シャープマーカーネオ
18           ボールドマーカーネオ
19              N-ZAP83
20              N-ZAP85
21              N-ZAP89
22        オクタシューター レプリカ
23             シャープマーカー
24             スプラシューター
25             わかばシューター
26            プライムシューター
27             ボールドマーカー
28            ボールドマーカー7
29              ボトルガイザー
30          スプラシューターコラボ
31             ホットブラスター
32           クラッシュブラスター
33         クラッシュブラスターネオ
34         ホットブラスターカスタム
35         ロングブラスターカスタム
36          ロングブラスターネクロ
37       ヒーローブラスター レプリカ
38        ノヴァブラスターベッチュー
39       ラピッドブラスターベッチュー
40             ノヴァブラスター
41           ノヴァ

In [4]:
# main

res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
trs = soup.find_all("tr")

In [5]:
main = []
sub = []
special = []
genre = []
range_= []
atack = []
rapid = []
vote = []
fav = []
for tr in trs[1:]:
    main.append(tr.find("span").text[:-1])
    sub.append(tr.find("span", class_="is-hide-small txt-small").text[1:])
    special.append(tr.find_all("span", class_="is-hide-small txt-small")[1].text[1:])
    genre.append(tr.find_all("td")[2].text)
    range_.append(tr.find_all("td")[3].text)
    atack.append(tr.find_all("td")[4].text)
    rapid.append(tr.find_all("td")[5].text)
    vote.append(tr.find_all("span", class_="js-count")[0].text)
    fav.append(tr.find_all("span", class_="js-count")[1].text)


In [6]:
main_df = pd.DataFrame({
    "main":main,
    "sub":sub,
    "special":special,
    "genre" : genre,
    "range" : range_,
    "atack" : atack,
    "rapid" : rapid,
    "vote" : vote,
    "fav" : fav
})
main_df.to_csv("../../data/Raw/main.csv", index=False)

In [7]:
#sub

res = requests.get(url+"sub/")
soup = BeautifulSoup(res.text, 'lxml')
trs = soup.find_all("tr")

In [8]:
sub = []
damage_max = []
damage_min = []
ink = []
num = []
good = []

for tr in trs[1:]:
    sub.append(tr.find("div", class_="txt").text[:-1])
    
    d = tr.find_all("td")[1].text.replace("\r\n", "").replace("-", "0").split(" ～ ")
    if(len(d) == 1):
        damage_min.append(d[0])
        damage_max.append(d[0])
    else:
        damage_min.append(d[0])
        damage_max.append(d[1])
    
    
    ink.append(tr.find_all("td")[2].text.replace("%", ""))
    num.append(tr.find_all("td")[3].text.replace("\r\n", "").replace("-", "0").replace("コ",""))
    good.append(tr.find_all("td")[4].text.replace("\n\n", "").replace(" ", ""))

In [9]:
sub_df = pd.DataFrame({
    "sub":sub,
    "damage_min": damage_min,
    "damage_max":damage_max,
    "ink" : ink,
    "install_num" : num,
    "good" : good
})
sub_df.to_csv("../../data/Raw/sub.csv", index=False)

In [10]:
#special

res = requests.get(url+"special/")
soup = BeautifulSoup(res.text, 'lxml')
trs = soup.find_all("tr")

In [11]:
special = []
damage_max = []
damage_min = []
duration = []
good = []

for tr in trs[1:]:
    special.append(tr.find_all("td")[0].text.replace("\n", "")[:-1])
    
    d = tr.find_all("td")[1].text.replace("\r\n", "").replace("-", "0").replace(" ", "").split("～")
    if len(d) == 1 or len(d) == 2 and len(d[1])==0:
        damage_min.append(d[0])
        damage_max.append(d[0])
    else:
        damage_min.append(d[0])
        damage_max.append(d[1])
    
    duration.append(tr.find_all("td")[2].text.replace("\r\n", "").replace("約","").replace("秒","").replace("-", "0"))
    good.append(tr.find_all("td")[4].text.replace("\n", "").replace(" ", ""))

In [12]:
special_df = pd.DataFrame({
    "special":special,
    "damage_min": damage_min,
    "damage_max":damage_max,
    "duration" : duration,
    "good" : good
})
special_df
special_df.to_csv("../../data/Raw/special.csv", index=False)